# Program 3.06: Instability of the model based on the complete equation

## Preamble

In [ ]:
using NBInclude

In [ ]:
@nbinclude("preamble/packages.ipynb");

In [ ]:
@nbinclude("preamble/functions.ipynb");

In [ ]:
model = ABM(3,
    
    # Global parameters
    model = Dict(   
    # Physical constantstransition
        :range => Float64,  
        :lambda=>Float64,
        :mu=>Float64,
        :adh=>Array{Float64},     
    # Division constants
        :tau_div => Float64,     
        :sigma_div => Float64,     
        :olap => Float64,
        :g_on=>Bool,
        :d_on=>Bool,
        :b=>Float64,
        :p=>Float64,      
        :q=>Float64,
        :k=>Float64,
    # Noise parameters
        :fp => Float64,
        :kp_on => Float64,
        :kp_off => Float64,
    # Reference values
        :t0=>Float64,
        :r0=>Float64,
        :f0=>Float64,
        :rep=>Float64,
        :atr=>Float64
    ),

    # Local parameters
    agent = Dict(     
        :t_div=>Float64,  
        :ni=>Float64,          
        :cell_state=>Int64,    
        # :m=>Float64,               
        :r=>Float64,           
        :vx=>Float64,          
        :vy=>Float64,
        :vz=>Float64,
        :vsumx=>Float64,
        :vsumy=>Float64,
        :vsumz=>Float64,
        :fx=>Float64,          
        :fy=>Float64,
        :fz=>Float64,
        :ni_a=>Float64,
        :r_ab=>Float64,        
        :r_bc=>Float64,
    # Noise
        :fpx=>Float64,          
        :fpy=>Float64,
        :fpz=>Float64,
        :marked=>Bool,
        :t_paired=>Float64
        ),
   

    agentODE = quote
    # Physical dynamics
        fx = 0; fy = 0; fz = 0
        vsumx = 0; vsumy = 0; vsumz = 0;
        ni = 0;
        @loopOverNeighbors it2 begin
            dij = CBMMetrics.euclidean(x,x[it2],y,y[it2],z,z[it2])
            if dij < mu*2*r && dij > 0
                if dij < 2*r
                    fx += rep * adh[cell_state, cell_state[it2]] * (2*r/dij-1) * (mu*2*r/dij-1) * (x-x[it2])/dij
                    fy += rep * adh[cell_state, cell_state[it2]] * (2*r/dij-1) * (mu*2*r/dij-1) * (y-y[it2])/dij
                    fz += rep * adh[cell_state, cell_state[it2]] * (2*r/dij-1) * (mu*2*r/dij-1) * (z-z[it2])/dij
                else
                    fx += atr * adh[cell_state, cell_state[it2]] * (2*r/dij-1) * (mu*2*r/dij-1) * (x-x[it2])/dij
                    fy += atr * adh[cell_state, cell_state[it2]] * (2*r/dij-1) * (mu*2*r/dij-1) * (y-y[it2])/dij
                    fz += atr * adh[cell_state, cell_state[it2]] * (2*r/dij-1) * (mu*2*r/dij-1) * (z-z[it2])/dij
                end
                vsumx += vx[it2]
                vsumy += vy[it2]
                vsumz += vz[it2]
            end
            if dij < range*2*r
                ni += 1
            end
        end
        if marked == true
            if t < t_paired
                fx += fpx
                fy += fpy
                fz += fpz
            else
                marked = false
            end
        end
        
        if ni > 0
            vx = (vsumx + fx/lambda) / ni
            vy = (vsumy + fy/lambda) / ni
            vz = (vsumz + fz/lambda) / ni
        else
            vx = 0
            vy = 0
            vz = 0
        end
    
        dt(x) = vx
        dt(y) = vy
        dt(z) = vz
    end,


    agentRule = quote
    # Growth
        if g_on
            if t > t_div
                x_div = CBMDistributions.normal(0,1) 
                y_div = CBMDistributions.normal(0,1) 
                z_div = CBMDistributions.normal(0,1)
                norm_div = sqrt(x_div^2+y_div^2+z_div^2)
                x_div /= norm_div
                y_div /= norm_div
                z_div /= norm_div
    
                r_sep = r*olap
                @addAgent(
                    x = x + r_sep * x_div,
                    y = y + r_sep * y_div,
                    z = z + r_sep * z_div,
                    # vx = 0,
                    # vy = 0,
                    # vz = 0,
                    vx = vx/2,
                    vy = vy/2,
                    vz = vz/2,
                    t_div = t + CBMDistributions.uniform(tau_div*(1-sigma_div),tau_div*(1+sigma_div))
                )
                @addAgent(
                    x = x - r_sep * x_div,
                    y = y - r_sep * y_div,
                    z = z - r_sep * z_div,
                    # vx = 0,
                    # vy = 0,
                    # vz = 0,
                    vx = vx/2,
                    vy = vy/2,
                    vz = vz/2,
                    t_div = t + CBMDistributions.uniform(tau_div*(1-sigma_div),tau_div*(1+sigma_div))
                )
                @removeAgent()
            end
        end

    # State evolution
        if d_on == true && cell_state != 3
            ni = 0
            ni_a = 0
            @loopOverNeighbors it2 begin
                dij = CBMMetrics.euclidean(x,x[it2],y,y[it2],z,z[it2])
                if dij < range*2*r
                    ni += 1
                    if(cell_state[it2] == 1)
                        ni_a +=1
                    end
                end
            end

            if ni != 0
                ni_a /= ni
            end

            ran = CBMDistributions.uniform(0,1)

            if cell_state == 1
                r_ab = p / (1 + k*ni_a)
                if ran < r_ab*dt
                    cell_state = 2
                end
            
            elseif cell_state == 2
                r_bc = q / (1 + k*ni_a)
                if ran < r_bc*dt
                    cell_state = 3
                end
            end   
        end
        
    end,


    agentAlg=CBMIntegrators.Heun()
);

## Initialization

In [ ]:
parameters = define_par(lambda=0.1);

dt = 0.0001;
save_each = round(Int64,0.001/dt);
n_cells = 4;

## Aggregate

In [ ]:
Random.seed!(1234)
com = initialize_growth(parameters;dt);

In [ ]:
grow_size!(com,save_each,n_cells)
# grow_time!(com,save_each,30)
m0 = length(com);

In [ ]:
println(com.N)
println(formed_correctly(com),"\n")
println(sum(com.vx))
println(sum(com.vy))
println(sum(com.vz),"\n")
println(mean(com.vsumx))
println(mean(com.vsumy))
println(mean(com.vsumz),"\n")
plot_aggregate(com, color_map, 1000, length(com))
# plot_aggregate_size(com, color_map, 60, 70, 5)

In [ ]:
find_instance(com, 7)
com[62].N

In [ ]:
stabilize!(com,save_each)
m0 = length(com);

In [ ]:
println(sum(com.vx))
println(sum(com.vy))
println(sum(com.vz),"\n")
println(mean(com.vsumx))
println(mean(com.vsumy))
println(mean(com.vsumz),"\n")
plot_aggregate(com, color_map, 1, length(com))

In [ ]:
# tmove = 40
# mechanics_evolve!(com,save_each,tmove)
# println(formed_correctly(com))
# plot_aggregate(com, color_map, m0, length(com))

In [ ]:
growncom = deepcopy(com);

## Differentiation

In [ ]:
com = deepcopy(growncom);

com.fp = 20
com.kp_on = 0.7
com.kp_off = 0.4;

In [ ]:
initialize_diff!(com);
# differentiate_all!(com,save_each,prot=false)
# differentiate!(com,save_each,40,prot=false)
differentiate!(com,save_each,40)
m2 = length(com);

In [ ]:
println(sum(com.vx))
println(sum(com.vy))
println(sum(com.vz),"\n")
println(mean(com.vsumx))
println(mean(com.vsumy))
println(mean(com.vsumz),"\n")
plot_aggregate(com, color_map, 660, m2)